# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [191]:
#Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
sns.set_style('darkgrid')
%matplotlib inline
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [192]:
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [193]:
#Your code here
num_clicks = len(set(df[df.action=='click']['id'].unique()))
num_views = len(set(df[df.action=='view']['id'].unique()))
num_views - num_clicks

4468

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [194]:
#Your code here
control_df = df[df['group'] == 'control']
control_df['action'].value_counts()

view     3332
click     932
Name: action, dtype: int64

In [195]:
experiment_df = df[df['group'] == 'experiment']
experiment_df['action'].value_counts()

view     2996
click     928
Name: action, dtype: int64

In [196]:
experiment_df[experiment_df['action'] == 'click'].id.count() 

928

In [197]:
(experiment_df['action'] == 'view').count()

3924

In [198]:
experiment_click_ret = ((experiment_df[experiment_df['action'] == 'click'].id.count()) / \
(experiment_df[experiment_df['action'] == 'view'].id.count()))
experiment_click_ret

0.3097463284379172

In [199]:
control_click_ret = ((control_df[control_df['action'] == 'click'].id.count()) / \
(control_df[control_df['action'] == 'view'].id.count()))
control_click_ret

0.2797118847539016

In [200]:
import scipy.stats as stats

In [201]:
df['count'] = 1
control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)

In [202]:
def welch_t(a, b):
    
    """ Calculate Welch's t-statistic for two samples. """
    numerator = a.mean() - b.mean()
    denominator = np.sqrt(a.var(ddof=1)/a.size + b.var(ddof=1)/b.size)
    
    return np.abs(numerator/denominator) # Return the t-score!

t_test = welch_t(control['click'], experiment['click'])
t_test

2.615440020788211

In [203]:
def welch_df(a, b):
    
    """ Calculate the effective degrees of freedom for two samples. """
    s1 = a.var(ddof=1) 
    s2 = b.var(ddof=1)
    n1 = a.size
    n2 = b.size
    
    numerator = (s1/n1 + s2/n2)**2
    denominator = (s1/ n1)**2/(n1 - 1) + (s2/ n2)**2/(n2 - 1)
    
    return numerator/denominator

df_test = welch_df(control['click'], experiment['click'])
df_test

6211.149291520621

In [204]:
p = 1 - stats.t.cdf(t_test, df_test)
p

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [206]:
#Your code here
click_control_rate = control.click.mean()
expected_experiment_clicks = click_control_rate * len(experiment)
expected_experiment_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [208]:
#Your code here
n = len(experiment)
p = click_control_rate
var = n * p * (1-p)
std = np.sqrt(var)
std

24.568547907005815

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [210]:
#Your code here
actual_experiment_clicks = experiment.click.sum()
z_score = (actual_experiment_clicks - expected_experiment_clicks)/std
z_score

3.6625360854823588

In [212]:
p_val = stats.norm.sf(z_score)
p_val

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Yes, both are well below 0.05**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.